### <u>Techniques et Outils pour la Preuve de Concept</u>: GrAMeFFSI (Graph Analysis Based Message Format and Field Semantics Inference For Binary Protocols, Using Recorded Network Traffic)

#### Referance du papier: 
G. Ládi, L. Buttyán et T. Holczer,  
**“Message Format and Field Semantics Inference for Binary Protocols Using Recorded Network Traffic,”**  
*2018 26th International Conference on Software, Telecommunications and Computer Networks (SoftCOM)*,  
Split, Croatia, 2018, pp. 1–6.  
DOI: [https://doi.org/10.23919/SOFTCOM.2018.8555813](https://doi.org/10.23919/SOFTCOM.2018.8555813)

### <center><u><h2>Introduction au problème et à la modélisation</h2></u></center>
Les protocoles binaires propriétaires sont aujourd'hui largement utilisés dans des domaines tels que l'industrie, l'IoT ou les systèmes embarqués, mais leurs spécifications ne sont pas toujours rendues publiques, ce qui complique l’analyse de sécurité. Pour analyser ou sécuriser ces systèmes, il faut comprendre la structure interne des messages (champs, longueur, type, signification), mais sans avoir la spécification du protocole. Afin de résoudre ce problème, l'algorithme GrAMeFFSI, proposé dans cet article, vise à inférer automatiquement les types de messages et la sémantique de certains champs d’un protocole binaire à partir de simples captures réseau, sans accès au code ni aux binaires.

Dans ce projet, l’objectif est de reproduire l’approche GrAMeFFSI sur un jeu de données existant, de comparer les modèles inférés à une spécification de référence du protocole étudié, puis d’analyser les performances et la reproductibilité de la publication originale.

### 1- Critères d’évaluation de la solution 
La qualité d’une solution de reverse engineering de protocoles est mesurée par trois métriques classiques : exactitude, concision et couverture. GrAMeFFSI ajoute deux métriques spécifiques pour la sémantique des champs : précision et précision ajustée.
* <u>exactitude</u> : proportion de types de messages inférés qui correspondent réellement à un type de message valide ;
* <u>concision</u> : degré de duplication des modèles, approximativement ;
* <u>couverture</u> : part des types de messages réels qui sont présents dans les modèles inférés ;
* <u>précision</u> : proportion de champs dont la sémantique (constante, compteur, longueur, chaîne, etc.) est correctement inférée, calculée à l’aide d’une distance d’édition d’arbres entre modèle vrai et modèle inféré ;
* <u>précision ajustée</u> : même calcul, mais en tolérant certaines erreurs dues à un trafic peu varié (par exemple un compteur vu comme constante si la valeur haute n’apparaît jamais);


### 2- Jeu de données de référence
L’article évalue l’algorithme GrAMeFFSI sur des traces Modbus et MQTT obtenues dans un environnement contrôlé, avec une génération de trafic riche et varié. Pour Modbus, les auteurs capturent des requêtes ou réponses entre un client et un serveur, puis comparent les modèles inférés à la spécification officielle Modbus. Pour MQTT, ils utilisent un broker Eclipse Mosquitto et un client Web HiveMQ, et couvrent les 14 types de messages définis par le standard, pour environ 1200 paquets.

Pour notre cas de preuve de concept, le jeu de données utilisé provient d’un dépôt GitHub existant qui utilise des captures réseau similaires au protocole TCP dont nous avons besoin pour notre programme. Le dataset consiste en un fichier .pcap contenant N paquets capturés lors d’échanges entre un client et un serveur.
* Protocole ciblé :
Dans notre cas d'etude, le protocole étudié est le Modbus/TCP. Sa spécification de référence est disponible dans [(https://github.com/ITI/ICS-Security-Tools/blob/master/pcaps/ModbusTCP/README.md)] et elle nous servira de base au modèle.


### 3- Résolution du problème de modélisation
L’algorithme GrAMeFFSI se subdivise en cinq phases : 
* préparation du jeu de donne;
* construction des arbres;
* fusion des modèles;
* optimisations;
*puis énumération des types de messages et des sémantiques de champs. 
Chaque arbre est un graphe orienté acyclique enraciné où chaque nœud représente un champ identifié (constante, compteur, longueur, chaîne, type énuméré, champ très variable).


#### Étape 1: Script pour lire le fichier .PCAP et extraire les payloads TCP Modbus
Cette étape initiale consiste à nettoyer et à normaliser les données capturées (fichiers PCAP). L'objectif est de s'assurer que l'algorithme travaille sur des flux de données cohérents en filtrant le bruit réseau et en uniformisant les adresses et ports sources/destinations, afin que tous les messages d'un même protocole soient traités comme un ensemble homogène.

In [19]:
from scapy.all import rdpcap, TCP
import pandas as pd

PCAP_FILE = "Modbus_capture.pcap"  # adapte si l'extension est différente
MODBUS_PORT = 502  # port Modbus/TCP standard

def extract_modbus_messages(pcap_file):
    packets = rdpcap(pcap_file)
    messages = []

    for pkt in packets:
        # On garde uniquement les paquets TCP avec du Modbus (port 502 client ou serveur)
        if TCP in pkt and (pkt[TCP].sport == MODBUS_PORT or pkt[TCP].dport == MODBUS_PORT):
            payload_bytes = bytes(pkt[TCP].payload)
            if len(payload_bytes) == 0:
                continue  # pas de données applicatives

            # Clé de flux : (IP_src, port_src, IP_dst, port_dst)
            try:
                ip_layer = pkt["IP"]
                flow_key = (
                    ip_layer.src,
                    pkt[TCP].sport,
                    ip_layer.dst,
                    pkt[TCP].dport,
                )
            except Exception:
                # Ignore paquets bizarres sans IP (par ex. IPv6 si pas géré ici)
                continue

            messages.append({
                "flow": flow_key,
                "length": len(payload_bytes),
                "payload": payload_bytes,
            })

    # DataFrame pratique pour la suite
    df = pd.DataFrame(messages)
    return df

df = extract_modbus_messages(PCAP_FILE)
print("Nombre de messages Modbus trouvés :", len(df))
print(df.head())
# Sauvegarde brute pour debug / étape suivante
df.to_pickle("modbus_messages.pkl")  # pour recharger facilement plus tard


Nombre de messages Modbus trouvés : 13390
                                     flow  length  \
0  (141.81.0.86, 502, 141.81.0.10, 57184)       6   
1  (141.81.0.10, 57184, 141.81.0.86, 502)      12   
2  (141.81.0.86, 502, 141.81.0.10, 57184)     273   
3  (141.81.0.10, 57184, 141.81.0.86, 502)      12   
4  (141.81.0.86, 502, 141.81.0.10, 57184)      26   

                                             payload  
0                           b'|\xf6\x00\x00\x00\x07'  
1  b'\x00\x00\x00\x00\x00\x06\xff\x04\x08\xd2\x00...  
2  b'|\xfe\x00\x00\x00\xc9\xff\x04\xc6\x00\x00\x0...  
3   b'\x00\x01\x00\x00\x00\x06\xff\x02\x00c\x00\x1e'  
4  b'\x00\x00\x00\x00\x00\x07\xff\x04\x04\x00\x00...  


#### Etape 2: Construction d’un arbre par flux
Ici, l'algorithme parcourt chaque octet des messages pour construire un arbre de flux (Flow Tree). Chaque nœud de l'arbre représente un champ du protocole, et les branches illustrent les différentes valeurs possibles. Cette structure permet de visualiser la hiérarchie et les dépendances entre les champs de données.

In [29]:
import pandas as pd

INPUT_PICKLE = "modbus_messages.pkl"
OUTPUT_PICKLE = "modbus_flows.pkl"

def load_messages():
    df = pd.read_pickle(INPUT_PICKLE)
    # On trie un peu pour avoir un ordre stable (optionnel)
    df = df.sort_values(by=["flow", "length"]).reset_index(drop=True)
    return df

def group_by_flow(df):
    flows = {}

    for _, row in df.iterrows():
        flow_key = row["flow"]          # tuple (src_ip, sport, dst_ip, dport)
        payload = row["payload"]        # type bytes
        if flow_key not in flows:
            flows[flow_key] = []
        # Chaque message est une liste d'octets, ce que l’algorithme considère comme une "séquence"
        flows[flow_key].append(list(payload))

    return flows


df = load_messages()
flows = group_by_flow(df)

print("Nombre de flux trouvés :", len(flows))
# Affichage d'exemple : 1er flux, 1er message (tronqué)
first_flow = next(iter(flows))
print("Premier flux :", first_flow)
print("Nombre de messages dans ce flux :", len(flows[first_flow]))
print("Premier message (longueur) :", len(flows[first_flow][0]))
print("Premiers octets :", flows[first_flow][0][:16])

# Sauvegarde pour les étapes suivantes (construction des arbres)
pd.to_pickle(flows, OUTPUT_PICKLE)
print("Sauvegardé dans", OUTPUT_PICKLE)


Nombre de flux trouvés : 28
Premier flux : ('141.81.0.10', 50594, '141.81.0.84', 502)
Nombre de messages dans ce flux : 530
Premier message (longueur) : 12
Premiers octets : [25, 137, 0, 0, 0, 6, 255, 4, 0, 48, 0, 40]
Sauvegardé dans modbus_flows.pkl


#### Etape 3: 	Fusion des modèles
Afin de simplifier l'arbre et d'identifier les structures répétitives, dans cette étape nous fusionnons les nœuds qui présentent des caractéristiques similaires. Cette étape est cruciale pour passer d'une simple représentation des messages capturés à un modèle généralisé capable de décrire le format global du protocole.

In [ ]:
import pandas as pd
import networkx as nx

INPUT_FLOWS = "modbus_flows.pkl"
OUTPUT_TREES = "modbus_trees.pkl"

def load_flows():
    flows = pd.read_pickle(INPUT_FLOWS)
    return flows  

def build_prefix_tree_for_flow(messages):
    G = nx.DiGraph()
    root = 0
    G.add_node(root, byte=None, count=0)  

    next_node_id = 1

    for msg in messages:
        current = root
        G.nodes[current]["count"] += 1  # ce nombre de messages passe par ce nœud

        for b in msg:
            child = None
            for succ in G.successors(current):
                if G.nodes[succ].get("byte") == b:
                    child = succ
                    break

            if child is None:
                child = next_node_id
                next_node_id += 1
                G.add_node(child, byte=b, count=0)
                G.add_edge(current, child)

            current = child
            G.nodes[current]["count"] += 1

    return G, root

def build_trees_for_all_flows(flows):
    trees = {}
    for flow_key, messages in flows.items():
        print(f"Construction de l'arbre pour le flux {flow_key} (messages: {len(messages)})")
        G, root = build_prefix_tree_for_flow(messages)
        trees[flow_key] = (G, root)
    return trees


flows = load_flows()
trees = build_trees_for_all_flows(flows)

# Exemple d'inspection : 1er flux
first_flow = next(iter(trees))
G, root = trees[first_flow]
print("Premier flux :", first_flow)
print("Nombre de noeuds dans son arbre :", G.number_of_nodes())
print("Nombre d'arêtes dans son arbre :", G.number_of_edges())
print("Successeurs directs de la racine :",
    [(n, G.nodes[n].get("byte"), G.nodes[n].get("count")) for n in G.successors(root)][:10])

# Sauvegarde pour les phases suivantes (fusion / optimisation)
pd.to_pickle(trees, OUTPUT_TREES)
print("Sauvegardé dans", OUTPUT_TREES)


Construction de l'arbre pour le flux ('141.81.0.10', 50594, '141.81.0.84', 502) (messages: 530)
Construction de l'arbre pour le flux ('141.81.0.10', 51411, '141.81.0.26', 502) (messages: 363)
Construction de l'arbre pour le flux ('141.81.0.10', 53414, '141.81.0.44', 502) (messages: 480)
Construction de l'arbre pour le flux ('141.81.0.10', 54138, '141.81.0.66', 502) (messages: 535)
Construction de l'arbre pour le flux ('141.81.0.10', 57184, '141.81.0.86', 502) (messages: 555)
Construction de l'arbre pour le flux ('141.81.0.10', 59598, '141.81.0.163', 502) (messages: 426)
Construction de l'arbre pour le flux ('141.81.0.10', 59599, '141.81.0.143', 502) (messages: 382)
Construction de l'arbre pour le flux ('141.81.0.10', 59758, '141.81.0.46', 502) (messages: 197)
Construction de l'arbre pour le flux ('141.81.0.10', 59796, '141.81.0.46', 502) (messages: 88)
Construction de l'arbre pour le flux ('141.81.0.10', 64338, '141.81.0.24', 502) (messages: 544)
Construction de l'arbre pour le flux ('

#### Etape 4: 	Phase d'Optimisation
Cette étape permet d'affiner le modèle en appliquant des heuristiques spécifiques pour corriger les erreurs d'interprétation courantes. Elle traite notamment deux cas complexes : la détection des champs de longueur variable (qui peuvent décaler la structure) et la suppression des faux types énumérés qui pourraient apparaître.

In [ ]:
import pandas as pd
import networkx as nx

INPUT_TREES = "modbus_trees.pkl"
OUTPUT_TREES = "modbus_trees_tagged.pkl"

THRESHOLD = 0.95  # seuil pour considérer un octet comme constant

def load_trees():
    trees = pd.read_pickle(INPUT_TREES)
    return trees  

def mark_node_types(G, root, threshold=THRESHOLD):
   
    G.nodes[root]["type"] = "root"
    G.nodes[root]["ratio"] = 1.0

    for parent, child in G.edges():
        parent_count = G.nodes[parent].get("count", 0)
        child_count = G.nodes[child].get("count", 0)

        if parent_count == 0:
            ratio = 0.0
        else:
            ratio = child_count / parent_count

        G.nodes[child]["ratio"] = ratio
        if ratio >= threshold:
            G.nodes[child]["type"] = "constant"
        else:
            G.nodes[child]["type"] = "variable"

def process_all_trees(trees, threshold=THRESHOLD):
    for flow_key, (G, root) in trees.items():
        print(f"Marquage des noeuds pour le flux {flow_key}")
        mark_node_types(G, root, threshold)
    return trees

if __name__ == "__main__":
    trees = load_trees()
    trees_tagged = process_all_trees(trees, THRESHOLD)

    # Exemple d'inspection sur le premier flux
    first_flow = next(iter(trees_tagged))
    G, root = trees_tagged[first_flow]
    print("Premier flux :", first_flow)
    print("Exemples de successeurs de la racine avec types :")
    examples = []
    for child in G.successors(root):
        node_data = G.nodes[child]
        examples.append(
            (child, node_data.get("byte"), node_data.get("count"),
             round(node_data.get("ratio", 0.0), 3),
             node_data.get("type"))
        )
        if len(examples) >= 10:
            break
    for e in examples:
        print(e)

    # Sauvegarde
    pd.to_pickle(trees_tagged, OUTPUT_TREES)
    print("Sauvegardé dans", OUTPUT_TREES)


Marquage des noeuds pour le flux ('141.81.0.10', 50594, '141.81.0.84', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 51411, '141.81.0.26', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 53414, '141.81.0.44', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 54138, '141.81.0.66', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 57184, '141.81.0.86', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 59598, '141.81.0.163', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 59599, '141.81.0.143', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 59758, '141.81.0.46', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 59796, '141.81.0.46', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 64338, '141.81.0.24', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 64340, '141.81.0.104', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 64341, '141.81.0.144', 502)
Marquage des noeuds pour le flux ('141.81.0.10', 64342, '141.81.0.164', 502)
Marquag

### Etape 5: Phase d'Interprétation Sémantique (Field Semantics Interpretation)
La dernière étape consiste à donner un sens aux données identifiées. L'algorithme analyse les valeurs des champs pour leur attribuer un rôle sémantique : constante (ID de protocole), compteur (numéro de séquence), longueur de charge utile ou encore chaînes de caractères.

In [ ]:
import pandas as pd
import networkx as nx
from collections import defaultdict, deque

INPUT_TREES_TAGGED = "modbus_trees_tagged.pkl"

def load_tagged_trees():
    trees = pd.read_pickle(INPUT_TREES_TAGGED)
    return trees  

def compute_depths(G, root):
    depths = {root: 0}
    queue = deque([root])

    while queue:
        current = queue.popleft()
        current_depth = depths[current]
        for child in G.successors(current):
            if child not in depths:
                depths[child] = current_depth + 1
                queue.append(child)

    return depths

def summarize_constants_by_depth(G, root):
    depths = compute_depths(G, root)

    const_bytes_per_depth = defaultdict(set)
    has_node_at_depth = defaultdict(bool)

    for node, depth in depths.items():
        node_type = G.nodes[node].get("type")
        byte_val = G.nodes[node].get("byte")

        if depth == 0:
            continue  # racine

        has_node_at_depth[depth] = True

        if node_type == "constant" and byte_val is not None:
            const_bytes_per_depth[depth].add(byte_val)

    # Préparer un résumé ordonné par profondeur
    summary = []
    for depth in sorted(has_node_at_depth.keys()):
        const_bytes = const_bytes_per_depth.get(depth, set())
        if const_bytes:
            label = "C"
        else:
            label = "V"
        const_hex = [f"0x{b:02X}" for b in sorted(const_bytes)]
        summary.append((depth, label, const_hex))

    return summary

def print_summary_for_flow(flow_key, G, root):
    print("Flux analysé :", flow_key)
    summary = summarize_constants_by_depth(G, root)
    print("Position | C/V | Octets constants (hex)")
    for depth, label, const_hex in summary:
        print(f"{depth:8d} |  {label}  | {', '.join(const_hex) if const_hex else '-'}")


trees_tagged = load_tagged_trees()

# Choix d'un flux à analyser : ici le premier par défaut
first_flow = next(iter(trees_tagged))
G, root = trees_tagged[first_flow]

print_summary_for_flow(first_flow, G, root)


Flux analysé : ('141.81.0.10', 50594, '141.81.0.84', 502)
Position | C/V | Octets constants (hex)
       1 |  V  | -
       2 |  V  | -
       3 |  C  | 0x00
       4 |  C  | 0x00
       5 |  C  | 0x00
       6 |  C  | 0x06, 0x08
       7 |  C  | 0xFF
       8 |  C  | 0x01, 0x02, 0x04, 0x0F
       9 |  C  | 0x00, 0x04
      10 |  C  | 0x00, 0x05, 0x30, 0x4C, 0xCB
      11 |  C  | 0x00
      12 |  C  | 0x01, 0x07, 0x0A, 0x1E, 0x28, 0x73
      13 |  C  | 0x01, 0x19, 0x1A, 0x1B
      14 |  C  | 0x00, 0x01, 0x07, 0x12, 0x13, 0x1D, 0x27, 0x28, 0x31, 0x3B, 0x45, 0x53, 0x5D, 0x60, 0x6A, 0x6E, 0x76, 0x7B, 0x87, 0x8B, 0x91, 0x93, 0x97, 0xA1, 0xA7, 0xAD, 0xB4, 0xB6, 0xBA, 0xC3, 0xC5, 0xCE, 0xD0, 0xD9, 0xDC, 0xE6, 0xE7, 0xE8, 0xF1, 0xF8, 0xFC
      15 |  C  | 0x00
      16 |  C  | 0x00
      17 |  C  | 0x00
      18 |  C  | 0x06
      19 |  C  | 0xFF
      20 |  C  | 0x04
      21 |  C  | 0x05
      22 |  C  | 0x14
      23 |  C  | 0x00
      24 |  C  | 0x04
      25 |  C  | 0x19, 0x1A, 0x1B
    

## construction de l'arbre global

In [28]:
import pandas as pd
import networkx as nx
from collections import deque

FLOWS_PICKLE = "modbus_flows.pkl"
TREES_TAGGED_PICKLE = "modbus_trees_tagged.pkl"

OUTPUT_GLOBAL_DOT = "modbus_global_tree.dot"

THRESHOLD = 0.95


def load_flows():
    return pd.read_pickle(FLOWS_PICKLE)  # dict: flow_key -> list[list[int]]


def build_global_tree(flows):
    G = nx.DiGraph()
    root = 0
    G.add_node(root, byte=None, count=0)

    next_node_id = 1

    for flow_key, messages in flows.items():
        print(f"Ajout des messages du flux {flow_key} (messages: {len(messages)})")
        for msg in messages:
            current = root
            G.nodes[current]["count"] += 1

            for b in msg:
                child = None
                for succ in G.successors(current):
                    if G.nodes[succ].get("byte") == b:
                        child = succ
                        break

                if child is None:
                    child = next_node_id
                    next_node_id += 1
                    G.add_node(child, byte=b, count=0)
                    G.add_edge(current, child)

                current = child
                G.nodes[current]["count"] += 1

    return G, root


def mark_node_types(G, root, threshold=THRESHOLD):
    G.nodes[root]["type"] = "root"
    G.nodes[root]["ratio"] = 1.0

    for parent, child in G.edges():
        parent_count = G.nodes[parent].get("count", 0)
        child_count = G.nodes[child].get("count", 0)

        ratio = child_count / parent_count if parent_count > 0 else 0.0
        G.nodes[child]["ratio"] = ratio

        if ratio >= threshold:
            G.nodes[child]["type"] = "constant"
        else:
            G.nodes[child]["type"] = "variable"


def export_global_tree_dot(G, root, dot_path):
    dot = nx.DiGraph()

    for node in G.nodes():
        data = G.nodes[node]

        if node == root:
            label = "ROOT"
            color = "black"
            shape = "doublecircle"
        else:
            byte = data.get("byte")
            count = data.get("count", 0)
            ratio = data.get("ratio", 0.0)
            ntype = data.get("type", "unknown")

            label = f"byte={byte}\\ncount={count}\\nratio={ratio:.2f}\\n{ntype}"

            color = "green" if ntype == "constant" else "red"
            shape = "circle"

        dot.add_node(
            node,
            label=label,
            color=color,
            shape=shape
        )

    for u, v in G.edges():
        dot.add_edge(u, v)

    nx.drawing.nx_pydot.write_dot(dot, dot_path)
    print(f"Arbre global exporté au format DOT dans {dot_path}")



flows = load_flows()

print("Construction de l'arbre global...")
G_global, root_global = build_global_tree(flows)
print("Nombre de noeuds :", G_global.number_of_nodes())
print("Nombre d'arêtes :", G_global.number_of_edges())

print("Marquage constant / variable...")
mark_node_types(G_global, root_global, THRESHOLD)

depths = compute_depths(G_global, root_global)
print("Niveau 1 (après la racine) :")
for n in G_global.nodes():
    if depths.get(n) == 1:
        print(
            n,
            G_global.nodes[n].get("byte"),
            G_global.nodes[n].get("count"),
            round(G_global.nodes[n].get("ratio", 0.0), 3),
            G_global.nodes[n].get("type")
            )

pd.to_pickle((G_global, root_global), OUTPUT_GLOBAL_PICKLE)
print("Arbre global sauvegardé dans", OUTPUT_GLOBAL_PICKLE)
    
export_global_tree_dot(G_global, root_global, OUTPUT_GLOBAL_DOT)


Construction de l'arbre global...
Ajout des messages du flux ('141.81.0.10', 50594, '141.81.0.84', 502) (messages: 530)
Ajout des messages du flux ('141.81.0.10', 51411, '141.81.0.26', 502) (messages: 363)
Ajout des messages du flux ('141.81.0.10', 53414, '141.81.0.44', 502) (messages: 480)
Ajout des messages du flux ('141.81.0.10', 54138, '141.81.0.66', 502) (messages: 535)
Ajout des messages du flux ('141.81.0.10', 57184, '141.81.0.86', 502) (messages: 555)
Ajout des messages du flux ('141.81.0.10', 59598, '141.81.0.163', 502) (messages: 426)
Ajout des messages du flux ('141.81.0.10', 59599, '141.81.0.143', 502) (messages: 382)
Ajout des messages du flux ('141.81.0.10', 59758, '141.81.0.46', 502) (messages: 197)
Ajout des messages du flux ('141.81.0.10', 59796, '141.81.0.46', 502) (messages: 88)
Ajout des messages du flux ('141.81.0.10', 64338, '141.81.0.24', 502) (messages: 544)
Ajout des messages du flux ('141.81.0.10', 64340, '141.81.0.104', 502) (messages: 495)
Ajout des messages

### 4- Scénarios et cas d'utilisation (reproductibles)
Dans l'article, les auteurs présentent deux scénarios bien précis :

Le premier scénario de validation repose sur l'étude du protocole industriel Modbus, mis en œuvre au sein d'un banc d'essai dédié aux systèmes de contrôle industriel (SCI). Pour générer un jeu de données pertinent, une série d'opérations de lecture et d'écriture a été effectuée manuellement de manière répétée, en utilisant une vaste gamme de paramètres conformes aux spécifications légales du protocole. Cette méthodologie a permis la collecte d'un échantillon massif d'environ 20 000 paires de requêtes-réponses. Afin de garantir la reproductibilité de l'expérience et la cohérence de l'analyse, l'outil editcap de la suite Wireshark a été utilisé pour uniformiser les ports sources et destinations, permettant ainsi à l'algorithme de traiter l'ensemble des paquets comme un flux unique et structuré.

Le second scénario se concentre sur le protocole MQTT, pilier de l'Internet des objets (IoT), testé dans un environnement contrôlé utilisant le serveur open source Eclipse Mosquitto et le client HiveMQ WebSocket. L'objectif de cette manipulation était de couvrir le spectre fonctionnel le plus large possible en exécutant un maximum d'opérations avec diverses combinaisons de paramètres. Le trafic ainsi généré a été capturé directement sur le serveur via Wireshark, aboutissant à un échantillon de 1 200 paquets. Ce cas d'utilisation démontre la capacité de la méthode GrAMeFFSI à s'adapter à des protocoles modernes, plus légers mais structurellement différents des protocoles industriels classiques.

### 5- Avantages de l’approche proposée
Les résultats expérimentaux présentés dans la publication montrent que l’approche GrAMeFFSI atteint une correction et une couverture parfaites, égales à 1.0, sur l’ensemble des protocoles testés par les auteurs. Cela signifie que tous les champs pertinents ont été correctement identifiés sans générer de faux positifs dans les scénarios évalués. Ces performances élevées démontrent l’efficacité de l’approche lorsque les conditions expérimentales sont favorables et que les données sont suffisamment représentatives.

Un autre avantage majeur de GrAMeFFSI réside dans l’utilisation d’une modélisation par graphes pour représenter la structure des messages. Cette représentation permet de distinguer naturellement des types de messages mutuellement exclusifs, c’est-à-dire des messages partageant un même protocole mais correspondant à des fonctions ou commandes différentes. Grâce à l’analyse des chemins et des sous-graphes, l’algorithme parvient à isoler ces structures sans connaissance préalable du protocole.

Enfin, les auteurs montrent que, lorsque les traces réseau sont de haute qualité (grand nombre de messages, faible bruit, structure stable), l’exactitude ajustée dépasse 95 %. Ce résultat confirme que la méthode est particulièrement bien adaptée à des environnements industriels ou embarqués où les communications sont régulières et répétitives, comme les protocoles SCADA, IoT ou automobiles.

#### <u>Limites identifiées de la méthode</u>

Une première limite importante concerne le chiffrement des communications. L’approche GrAMeFFSI repose sur l’analyse statistique et structurelle des octets observés dans les messages. Lorsque le trafic est chiffré ou obfusqué, cette structure disparaît, rendant impossible toute inférence du format ou de la sémantique des champs. Cette limitation est inhérente à toutes les méthodes d’analyse passive de protocoles binaires.

La qualité et la quantité des données d’entrée constituent également un facteur critique. Les auteurs soulignent que, lorsque le nombre de messages observés est insuffisant ou que les traces sont bruitées, l’algorithme peut produire des erreurs d’inférence. Par exemple, des champs de longueur ou de type peuvent être incorrectement détectés comme constants si leur variabilité n’apparaît pas dans les données disponibles. Cette dépendance aux données souligne la nécessité de disposer de traces longues et représentatives pour garantir des résultats fiables.

Enfin, GrAMeFFSI ne prend pas en charge les champs dits split-byte, c’est-à-dire les champs encodés sur quelques bits seulement à l’intérieur d’un octet. Ce type d’encodage est courant dans certains protocoles comme MQTT, où plusieurs drapeaux binaires sont regroupés dans un même octet. L’algorithme, basé sur une analyse octet par octet, ne peut pas actuellement détecter ni interpréter correctement ce genre de structure fine.

<u>Commentaires sur la publication</u>

<u>Points positifs</u> : L’un des principaux atouts de la publication réside dans le choix de protocoles ouverts et largement documentés, tels que (Modbus, MQTT) combine a des outils accessibles (Wireshark), garantissent une excellente reproductibilité. La formalisation mathématique rigoureuse des métriques (ratios, fréquences) rend l'algorithme transparent et facilite son implémentation.

<u>Hypothèses omises et limites</u> :
La publication repose implicitement sur plusieurs hypothèses fortes qui ne sont pas explicitement discutées.Nous avons entre autre: 

* Qualité des données : L'article suppose des captures parfaites. En réalité, les pertes ou duplications de paquets pourraient fausser les statistiques de classification.

* Arbitraire des seuils : Le seuil d'énumération (8-20) est empirique. L'absence de méthode systématique pour l'ajuster limite la robustesse de l'approche face à des protocoles totalement inconnus ou des jeux de données restreints.

### Conclusion
Ce projet nous a permis de mettre en œuvre l'approche GrAMeFFSI, une méthode d'inférence automatique de protocoles binaires basée sur l'analyse de graphes orientés. L'objectif principal était de reconstruire la structure et la sémantique de protocoles inconnus à partir de traces réseau, sans connaissance préalable, en distinguant les champs constants des variables via l'analyse de fréquences.
La démarche a consisté à traduire les concepts théoriques de l'article en une implémentation Python appliquée au protocole Modbus/TCP. L'algorithme que nous avons développé assure l'extraction des données depuis des fichiers .pcap, le regroupement par flux, la construction d'un arbre de format global et la classification automatisée des nœuds.
Les résultats valident l'efficacité de l'analyse par graphes sur des flux non chiffrés, permettant une identification précise des invariants structurels. Toutefois, l'étude confirme également les limites intrinsèques à cette méthode, notamment sa sensibilité à la qualité du jeu de données et son inopérabilité face au chiffrement. Ce travail offre ainsi une base solide pour explorer des problématiques plus complexes, comme la détection de champs codés sur quelques bits ou l'analyse d'autres protocoles industriels.